In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'



In [4]:
orders = pd.read_csv("orders_20190822.csv",sep=";",decimal=",",parse_dates=["o_date"])

In [11]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
id_o       int64
user_id    int64
price      float64
o_date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 61.1 MB


In [68]:
orders["o_date"].max()

Timestamp('2017-12-31 00:00:00')

In [25]:
price_date = orders[["price","o_date"]]
price_date.head()

,price,o_date
0,539.0,2016-01-01
1,153.3,2016-01-01
2,55.3,2016-01-01
3,752.5,2016-01-01
4,4410.0,2016-01-01


In [26]:
price_date= price_date.set_index('o_date')

In [27]:
price_date.head()

,price
o_date,
2016-01-01,539.0
2016-01-01,153.3
2016-01-01,55.3
2016-01-01,752.5
2016-01-01,4410.0


In [51]:
y = price_date['price'].resample('MS').sum().astype(dtype="int64")

In [52]:
y.head()

o_date
2016-01-01    112520331
2016-02-01    100652262
2016-03-01    125415802
2016-04-01    135366050
2016-05-01    130065409
Freq: MS, Name: price, dtype: int64

In [57]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [58]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

D:\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]


ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:946.6453180443624
ARIMA(0, 0, 0)x(0, 1, 0, 12)12 - AIC:436.38265233093637
ARIMA(0, 0, 0)x(1, 0, 0, 12)12 - AIC:444.4756244538871
ARIMA(0, 0, 1)x(0, 0, 0, 12)12 - AIC:893.5468595479182
ARIMA(0, 0, 1)x(0, 1, 0, 12)12 - AIC:389.4518901237595
ARIMA(0, 0, 1)x(1, 0, 0, 12)12 - AIC:487.9810410841032
ARIMA(0, 1, 0)x(0, 0, 0, 12)12 - AIC:811.3986492226344
ARIMA(0, 1, 0)x(0, 1, 0, 12)12 - AIC:369.7587321193125
ARIMA(0, 1, 0)x(1, 0, 0, 12)12 - AIC:402.85277597880133
ARIMA(0, 1, 1)x(0, 0, 0, 12)12 - AIC:775.7670182700768
ARIMA(0, 1, 1)x(0, 1, 0, 12)12 - AIC:332.08440830134464
ARIMA(0, 1, 1)x(1, 0, 0, 12)12 - AIC:406.33572628832286
ARIMA(1, 0, 0)x(0, 0, 0, 12)12 - AIC:845.5540740887171
ARIMA(1, 0, 0)x(0, 1, 0, 12)12 - AIC:402.91981595984214
ARIMA(1, 0, 0)x(1, 0, 0, 12)12 - AIC:404.20870564074335
ARIMA(1, 0, 1)x(0, 0, 0, 12)12 - AIC:811.3308615221215
ARIMA(1, 0, 1)x(0, 1, 0, 12)12 - AIC:367.88048876997857
ARIMA(1, 0, 1)x(1, 0, 0, 12)12 - AIC:406.3358374348389
ARI

In [ ]:
ARIMA(0, 1, 1)x(0, 1, 0, 12)12 - AIC:332.08440830134464

In [60]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(0, 1, 1),
                                seasonal_order=(0, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.3095      0.158     -1.960      0.050      -0.619    3.84e-05
sigma2      1.959e+14   1.45e-16   1.35e+30      0.000    1.96e+14    1.96e+14


In [77]:
pred.predicted_mean.head(10)
pred_uc = results.get_forecast(steps=12)
forecast_result = pred_uc.predicted_mean.astype(dtype="int64")
forecast_result.index

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [ ]:
1801 256798898
1802 232640416
1803 267994924
1804 262849522
1805 276933049
1806 251486085
1807 250559778
1808 261724749
1809 276675505
1810 287647539
1811 363102609
1812 422386052

In [79]:
true_sales = pd.Series([256798898, 232640416, 267994924, 262849522, 276933049,251486085,250559778,261724749,276675505,287647539,363102609,422386052], index =['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'])
true_sales

2018-01-01    256798898
2018-02-01    232640416
2018-03-01    267994924
2018-04-01    262849522
2018-05-01    276933049
2018-06-01    251486085
2018-07-01    250559778
2018-08-01    261724749
2018-09-01    276675505
2018-10-01    287647539
2018-11-01    363102609
2018-12-01    422386052
dtype: int64

In [94]:
forecast_comparison = pd.concat([forecast_result, true_sales], axis=1)
forecast_comparison=forecast_comparison.rename(columns={0: "Предсказанное", 1: "Реальное"})
forecast_comparison

,Предсказанное,Реальное
2018-01-01,288716147,256798898
2018-02-01,263969693,232640416
2018-03-01,299648666,267994924
2018-04-01,287354680,262849522
2018-05-01,321926560,276933049
2018-06-01,299554600,251486085
2018-07-01,303466477,250559778
2018-08-01,308583062,261724749
2018-09-01,310080412,276675505
2018-10-01,353460108,287647539
